# TextRank algorithm from scratch

In [56]:
import numpy as np
import pandas as pd
import nltk
import re
# nltk.download('punkt')
import matplotlib.pyplot as plt 
import seaborn as sns
pd.set_option('display.max_colwidth',1000)

In [57]:
from expand_contractions import expand_contractions # from expanding contractions

https://github.com/hardikkamboj/Implementations-in-Python/blob/main/NLP/Expanding%20Contractions/expand_contractions.py

## Load Glove embeddings

In [58]:
import zipfile

GLOVE_DIR = 'data/glove/'
GLOVE_ZIP = GLOVE_DIR + 'glove.6B.50d.zip'

zip_ref = zipfile.ZipFile(GLOVE_ZIP, 'r')
zip_ref.extractall(GLOVE_DIR)
zip_ref.close()

In [59]:
import numpy as np

def load_glove_vectors(fn):
    print("Loading Glove Model")
    with open( fn,'r', encoding='utf8') as glove_vector_file:
        model = {}
        for line in glove_vector_file:
            parts = line.split()
            word = parts[0]
            embedding = np.array([float(val) for val in parts[1:]])
            model[word] = embedding
        print("Loaded {} words".format(len(model)))
    return model

glove_vectors = load_glove_vectors('data/glove/glove.6B.50d.txt')

Loading Glove Model
Loaded 400000 words


In [60]:
glove_vectors['car']

array([ 0.47685 , -0.084552,  1.4641  ,  0.047017,  0.14686 ,  0.5082  ,
       -1.2228  , -0.22607 ,  0.19306 , -0.29756 ,  0.20599 , -0.71284 ,
       -1.6288  ,  0.17096 ,  0.74797 , -0.061943, -0.65766 ,  1.3786  ,
       -0.68043 , -1.7551  ,  0.58319 ,  0.25157 , -1.2114  ,  0.81343 ,
        0.094825, -1.6819  , -0.64498 ,  0.6322  ,  1.1211  ,  0.16112 ,
        2.5379  ,  0.24852 , -0.26816 ,  0.32818 ,  1.2916  ,  0.23548 ,
        0.61465 , -0.1344  , -0.13237 ,  0.27398 , -0.11821 ,  0.1354  ,
        0.074306, -0.61951 ,  0.45472 , -0.30318 , -0.21883 , -0.56054 ,
        1.1177  , -0.36595 ])

In [61]:
len(list(glove_vectors.keys()))

400000

In [62]:
len(glove_vectors['car'])

50

Each word is represented as a vector of length 50

# Read articles 

In [63]:
articles = pd.read_csv("data/tennis_articles_v4.csv")
print(articles.shape)
articles.head(10)

(8, 3)


,article_id,article_text,source
0,1,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...",https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/
1,2,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours w...",http://www.tennis.com/pro-game/2018/10/copil-stuns-5th-ranked-zverev-to-reach-swiss-indoors-final/77721/
2,3,"Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition. Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment. ""They only left me three days to decide"", Federer said. ""I didn't to have time to consult with all the people I had to consult. ""I could not make a decision in that time, so I told them to do what they wanted."" The 20-time Grand Slam champion has voiced doubts about the wisdom of the one-week format to be introduced by organisers Kosmos, who have promised the International Tennis Federation up to $3 billion in prize money over the next quarter-century. The competition is set to feature 18 countries in the November 18-24 finals in Madrid next year, and will replace the classic hom...",https://scroll.in/field/899938/tennis-roger-federer-ignored-deadline-set-by-new-davis-cup
3,4,"Kei Nishikori will try to end his long losing streak in ATP finals and Kevin Anderson will go for his second title of the year at the Erste Bank Open on Sunday. The fifth-seeded Nishikori reached his third final of 2018 after beating Mikhail Kukushkin of Kazakhstan 6-4, 6-3 in the semifinals. A winner of 11 ATP events, Nishikori hasn't triumphed since winning in Memphis in February 2016. He has lost eight straight finals since. The second-seeded Anderson defeated Fernando Verdasco 6-3, 3-6, 6-4. Anderson has a shot at a fifth career title and second of the year after winning in New York in February. Nishikori leads And

In [64]:
#stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [65]:
from nltk.tokenize import sent_tokenize, word_tokenize

CLEAN_PATTERN = r'[^a-zA-z\s]'

def clean(word):
    return re.sub(CLEAN_PATTERN, '', word)

def clean_sentence(sentence):
    sentence = [clean(word) for word in sentence]
    return [word for word in sentence if word]

def clean_sentences(sentences):
    return [clean_sentence(sentence) for sentence in sentences]

def lower(sentence):
    return [word.lower() for word in sentence]

def remove_stopwords(sentence):
    words = [word for word in sentence if word not in stop_words]
    return [word for word in words if len(word) >0]

def tokenize_words(sentences):
    return [word_tokenize(sentence) 
              for sentence in sentences]

def fix_contractions(sentences):
    return [expand_contractions(sentence) for sentence in sentences]

In [66]:
articles['SentencesInArticle'] = articles.article_text.apply(sent_tokenize)
articles['WordsInSentences'] = articles.SentencesInArticle \
              .apply(fix_contractions)\
              .apply(lower)\
              .apply(tokenize_words)\
              .apply(remove_stopwords)\
              .apply(clean_sentences)

In [67]:
articles.head(2)

,article_id,article_text,source,SentencesInArticle,WordsInSentences
0,1,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...",https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/,"[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,..."
1,2,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel witho

In [68]:
articles = articles[['article_text','SentencesInArticle', 'WordsInSentences']] # selecting only the two cols
articles.head(2)

,article_text,SentencesInArticle,WordsInSentences
0,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...","[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,..."
1,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zver

In [69]:
VECTOR_SIZE = 50
EMPTY_VECTOR = np.zeros(VECTOR_SIZE)

def sentence_vector(sentence):
    return sum([glove_vectors.get(word, EMPTY_VECTOR) 
                  for word in sentence])/len(sentence)

def sentences_to_vectors(sentences):
    return [sentence_vector(sentence) 
              for sentence in sentences]

## Articles with sentence vectors

In [70]:
articles['SentenceVector'] =  articles.WordsInSentences.apply(sentences_to_vectors)

In [71]:
articles.head()

,article_text,SentencesInArticle,WordsInSentences,SentenceVector
0,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...","[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,...","[[0.07161230769230773, 0.5263353076923079, -0.22625407692307692, 0.4190584615384615, -0.0952153076923077, -0.10156600000000002, -0.40362538461538466, 0.400633476923077, -0.35831371000000006, -0.11500546153846154, 0.11607930769230769, -0.146259, -0.6847891923076924, 0.014327538461538456, 0.783284653846154, 0.046932615384615385, -0.10918245384615388, 0.054596461538461545, -0.29416438461538463, -0.1929796923076923, -0.2468181230769231, 0.3106658461538461, 0.011112153846153848, 0.30192715384615376, 0.12148384615384614, -1.328529230769231, -0.2920881538461539, -0.07721707692307692, -0.3258306923076923, -0.36910523076923074, 2.7283676923076925, 0.24652538461538465, -0.041018076923076914, 0.07203453846153846, 0.022849225384615382, -0.01111693076923078, 0.2808099230769231, 0.4716086153846154, -0.020941546153846143, -0.6126247692307694, -0.007031076923076909, -0.08967269230769231, 0.18087161538461535, 0.06510876923076923,

In [72]:
temp = articles.iloc[0][1]

In [73]:
len(temp) # it has 17 sentences

17

In [74]:
np.array(temp).shape

(17,)

In [75]:
temp_vectors = articles.iloc[0][2]
np.array(temp_vectors).shape 
# each sentence is represented a vector of size 50

(17,)

Each sentence is represented as a vector, which is average of all the glove vectors of the words present in a sentence. SO, now, each sentence can be represented as a vector of size 50

In [76]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity_matrix(sentence_vectors):
    sim_mat = np.zeros([len(sentence_vectors), len(sentence_vectors)])
    for i in range(len(sentence_vectors)):
        for j in range(len(sentence_vectors)):
            element_i = sentence_vectors[i].reshape(1,VECTOR_SIZE)
            element_j = sentence_vectors[j].reshape(1,VECTOR_SIZE)
            sim_mat[i][j] = cosine_similarity(element_i,
                                              element_j)[0,0]
    return sim_mat

Similarity matrix, for a sentece vectors of n sentences, is a square matrix of size (n,n), where the cell i,j tells how much similar the sentence i is to sentence j

In [77]:
articles['SimMatrix'] = \
     articles.SentenceVector.apply(similarity_matrix)

In [78]:
sim_matrix = articles['SimMatrix'][0]

In [79]:
np.array(sim_matrix).shape

(17, 17)

In [80]:
import networkx as nx

def compute_graph(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)
    return scores

# Graph

In [81]:
articles['Graph'] = articles.SimMatrix.apply(compute_graph)

In [82]:
articles.head()

,article_text,SentencesInArticle,WordsInSentences,SentenceVector,SimMatrix,Graph
0,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...","[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,...","[[0.07161230769230773, 0.5263353076923079, -0.22625407692307692, 0.4190584615384615, -0.0952153076923077, -0.10156600000000002, -0.40362538461538466, 0.400633476923077, -0.35831371000000006, -0.11500546153846154, 0.11607930769230769, -0.146259, -0.6847891923076924, 0.014327538461538456, 0.783284653846154, 0.046932615384615385, -0.10918245384615388, 0.054596461538461545, -0.29416438461538463, -0.1929796923076923, -0.2468181230769231, 0.3106658461538461, 0.011112153846153848, 0.30192715384615376, 0.12148384615384614, -1.328529230769231, -0.2920881538461539, -0.07721707692307692, -0.3258306923076923, -0.36910523076923074, 2.7283676923076925, 0.24652538461538465, -0.041018076923076914, 0.07203453846153846, 0.022849225384615382, -0.01111693076923078, 0.2808099230769231, 0.4716086153846154, -0.020941546153846143, -0.6126247692307694, -0.007031076923076909, -0.08967269230769231, 0.18087161538461535, 0.06

In [83]:
temp = articles['Graph'][0]

In [84]:
temp

{0: 0.05554199464753272,
 1: 0.05964251627873144,
 2: 0.05841053156540537,
 3: 0.05969349780006848,
 4: 0.05648586101195639,
 5: 0.05921860498291677,
 6: 0.05938610203482372,
 7: 0.05950432590883181,
 8: 0.05960892627588667,
 9: 0.05691782210622912,
 10: 0.059056937650340785,
 11: 0.05997064367483894,
 12: 0.05923363532028829,
 13: 0.05916500837372274,
 14: 0.05972978591413795,
 15: 0.059770390866579516,
 16: 0.05866341558770929}

# Ranked sentences 

In [85]:
def get_ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s) 
                         for i,s in enumerate(sentences)), 
                                reverse=True)
    top_n_sentences = [sentence 
                        for score,sentence in top_scores[:n]]
    return " ".join(top_n_sentences)

articles['Summary'] = articles.apply(lambda d: 
                                     get_ranked_sentences(d.SentencesInArticle, 
                                                          d.Graph), axis=1)

In [86]:
articles.head()

,article_text,SentencesInArticle,WordsInSentences,SentenceVector,SimMatrix,Graph,Summary
0,"Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the sa...","[Maria Sharapova has basically no friends as tennis players on the WTA Tour., The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much., I think everyone knows this is my job here., When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match., I'm a pretty competitive girl., I say my hellos, but I'm not sending any players flowers as well., Uhm, I'm not really friendly or close to many players., I have not a lot of friends away from the courts.', When she said she is not really close to a lot of players, is that something strategic that she is doing?, Is it different on the men's tour than the women's tour?, 'No, not at all., I think just because you'...","[[maria, sharapova, has, basically, no, friends, as, tennis, players, on, the, wta, tour], [the, russian, player, has, no, problems, in, openly, speaking, about, it, and, in, a, recent, interview, she, said, i, do, not, really, hide, any, feelings, too, much], [i, think, everyone, knows, this, is, my, job, here], [when, i, am, on, the, courts, or, when, i, am, on, the, court, playing, i, am, a, competitor, and, i, want, to, beat, every, single, person, whether, they, are, in, the, locker, room, or, across, the, netso, i, am, not, the, one, to, strike, up, a, conversation, about, the, weather, and, know, that, in, the, next, few, minutes, i, have, to, go, and, try, to, win, a, tennis, match], [i, am, a, pretty, competitive, girl], [i, say, my, hellos, but, i, am, not, sending, any, players, flowers, as, well], [uhm, i, am, not, really, friendly, or, close, to, many, players], [i, have, not, a, lot, of, friends, away, from, the, courts], [when, she, said, she, is, not, really, close,...","[[0.07161230769230773, 0.5263353076923079, -0.22625407692307692, 0.4190584615384615, -0.0952153076923077, -0.10156600000000002, -0.40362538461538466, 0.400633476923077, -0.35831371000000006, -0.11500546153846154, 0.11607930769230769, -0.146259, -0.6847891923076924, 0.014327538461538456, 0.783284653846154, 0.046932615384615385, -0.10918245384615388, 0.054596461538461545, -0.29416438461538463, -0.1929796923076923, -0.2468181230769231, 0.3106658461538461, 0.011112153846153848, 0.30192715384615376, 0.12148384615384614, -1.328529230769231, -0.2920881538461539, -0.07721707692307692, -0.3258306923076923, -0.36910523076923074, 2.7283676923076925, 0.24652538461538465, -0.041018076923076914, 0.07203453846153846, 0.022849225384615382, -0.01111693076923078, 0.2808099230769231, 0.4716086153846154, -0.020941546153846143, -0.6126247692307694, -0.007031076923076909, -0.08967269230769231, 0.180871615384615

In [87]:
def get_summary(idx):
    print('Original text - ')
    print(articles['article_text'][idx])
    print(' ')
    print('Summary - ')
    print(articles['Summary'])

In [90]:
get_summary(0)

Original text - 
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you